# Fantasy Football
 - Collecting data for player stats and cost
 - Player stats scraped from https://fantasyfootball.skysports.com/team-select
 - Player costs scraped from https://fantasyfootball.skysports.com/stats
 - Point method scraped from https://fantasyfootball.skysports.com/help

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
point = pd.read_csv("data/fantasy-scoring.csv")
gk_stats = pd.read_excel("data/skysports-stats.xlsx", sheet_name="GK")
def_stats = pd.read_excel("data/skysports-stats.xlsx", sheet_name="DEF")
mid_stats = pd.read_excel("data/skysports-stats.xlsx", sheet_name="MID")
str_stats = pd.read_excel("data/skysports-stats.xlsx", sheet_name="STR")
cost = pd.read_excel("data/skysports-stats.xlsx", sheet_name="COST")

## Converting Stats to Points
 - Column alignment
 - Missing stats for points
   - Penalty Save
   - Penalty Miss
   - 2+ Goal Conceded (GK & DEF)
   - Calculated points and skysports points might not align
 - Calculated points

In [3]:
point_to_stat = [
    'Gls', 'Cl Sh', 'Gls', 'Gls', 'Cl Sh', 
    'NA', 'MotM', 'Ass', 'Save 2', 'Tack 2',
    'Pass 2', 'Shot 2', 'App', 'Save 1', 'Tack 1',
    'Pass 1', 'Shot 1', 'Sub', 'Gl Con', 'Yel',
    'NA', 'NA', 'Red'
]
position_apply = [
    ['GK','DEF'], ['GK'], ['MID'], ['STR'], ['DEF'],
    ['GK'],['ALL'], ['ALL'], ['GK'], ['ALL'],
    ['ALL'], ['ALL'], ['ALL'], ['GK'], ['ALL'],
    ['ALL'], ['ALL'], ['ALL'], ['GK', 'DEF'], ['ALL'],
    ['ALL'], ['ALL'], ['ALL']
]
point['STAT'] = point_to_stat
point['POSITION'] = position_apply

In [4]:
def get_point_index(position, point):
    index = (point
             .POSITION
             .apply(lambda x: any([position in x, 'ALL' in x])))
    return index

def get_point_dict(point, index):
    df = point[index]
    df_dict = dict(zip(df.STAT, df.PTS))
    return df_dict

point_dict = {}
for position  in ['GK', 'DEF', 'MID', 'STR']:
    ix = get_point_index(position, point)
    point_dict[position] = get_point_dict(point, ix)
    
def sum_points(stats, position, pt_dict):
    keys = list(pt_dict[position].keys())
    keys = [k for k in keys if k != 'NA']
    output = pd.Series(np.zeros(stats.shape[0]))
    for k in keys:
        # print(k)
        pt_tmp = stats[k] * pt_dict[position][k]
        if k != 'Gl Con':
            pass
        else:
            pt_tmp /= 4
        output += pt_tmp
    return output

In [5]:
gk_stats['Calc Points'] = sum_points(gk_stats, 'GK', point_dict)
def_stats['Calc Points'] = sum_points(def_stats, 'DEF', point_dict)
mid_stats['Calc Points'] = sum_points(mid_stats, 'MID', point_dict)
str_stats['Calc Points'] = sum_points(str_stats, 'STR', point_dict)
gk_stats['Position'] = 'GK'
def_stats['Position'] = 'DEF'
mid_stats['Position'] = 'MID'
str_stats['Position'] = 'STR'

## Join Stats with Costs
 - Check if all players have cost and stats

In [6]:
players = cost.Player.apply(lambda x: x.replace(" ,", ","))

In [7]:
for df in [gk_stats, def_stats, mid_stats, str_stats]:
    check = all([ply in list(players) for ply in df['Player']])
    print(check)
    # looks good

True
True
True
True


In [8]:
cost['Player'] = cost.Player.apply(lambda x: x.replace(" ,", ","))
cost['Points'] = cost.Points.apply(lambda x: float(x.split(" ")[0]))
cost['Cost'] = cost.Cost.apply(lambda x: float(x.split("£")[1]))

In [9]:
stats = (gk_stats
         .append(def_stats)
         .append(mid_stats)
         .append(str_stats))

In [10]:
clean_data = cost.merge(stats, how="inner", on=['Player'])

In [11]:
def reorder_columns(df, first_columns):
    column_array = np.array(df.columns)
    first_array = np.array(first_columns)
    not_in_mask = np.isin(column_array, first_array, invert=True)
    reorder_col = first_columns + list(column_array[not_in_mask])
    return df[reorder_col]

In [12]:
first_col = ['Player','Position','Cost','Points', 'Calc Points']
clean_data = reorder_columns(clean_data, first_col)

In [13]:
clean_data.to_csv("data/clean-data.csv", index=False)